## Base Model: Hierarchical Multiobjective MIP

In [16]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal', 'Geothermal']

# Emission factors for each energy source (MTCO2e/GWh) 
E_factor_data = {
    'Wind': [0.000015, 0.000015, 0.000015],
    'Solar': [0.0000445, 0.0000445, 0.0000445],
    'Oil': [0.00049, 0.00049, 0.00049],
    'Nuclear': [0.000012, 0.000012, 0.000012],
    'Hydro': [0.000024, 0.000024, 0.000024],
    'Natural Gas': [0.00049, 0.00049, 0.00049],
    'Coal': [0.00082, 0.00082, 0.00082],
    'Geothermal': [0.000038, 0.000038, 0.000038]
}

# Electricity demand for each time period (GWh) 
D_data = {2025: 611806.94, 2030: 694406.68, 2035: 787841.11} 

# Emission reduction  for each time period (#MTCO2e) 
Goals = {2025: 35.19, 2030: 28.89, 2035: 0}  


# Fixed generation capacity for each time period (GWh)  
Fixed_GEN_data = {
    'Wind': [64389.48],
    'Solar': [12184.92],
    'Oil': [1379.91],
    'Nuclear': [78631.37],
    'Hydro': [402575.9],
    'Natural Gas': [106529.07],
    'Coal': [8184.54],
    'Geothermal': [8281.2]
}


# Cost data for each energy source (CAD per GWh) 
Cost_data = {
    'Wind': [80830, 80830, 80830],
    'Solar': [205500, 205500, 205500],
    'Oil': [61650, 61650, 61650],
    'Nuclear': [89050, 89050, 89050],
    'Hydro': [30140, 30140, 30140],
    'Natural Gas': [97270, 97270, 97270],
    'Coal': [83570, 83570, 83570],
    'Geothermal': [82200, 82200, 82200]
}

# Capacity increase costs for each power plant type (CAD)
Increase_Cost = {
    'Nuclear': [9.59e9, 9.59e9, 9.59e9],
    'Solar': [12.878e9, 12.878e9, 12.878e9],
    'Wind': [4.11e9, 4.11e9, 4.11e9]
}

# Capacity increase amounts for each power plant type (GWh)
Capacity_Added = {
    'Nuclear': [48180, 48180, 48180],
    'Solar': [139809.6, 139809.6, 139809.6],
    'Wind': [43800, 43800, 43800]
}

# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(T, vtype=GRB.CONTINUOUS, name="Deviation")

# Decision variables for capacity increase (only for Wind, Solar, Nuclear)
Capacity_Increase = m.addVars(['Wind', 'Solar', 'Nuclear'], T, vtype=GRB.CONTINUOUS, name="Capacity_Increase")

# Binary variables for capacity increase decision
Capacity_Increase_Decision = m.addVars(['Wind', 'Solar', 'Nuclear'],T,vtype=GRB.BINARY, name="CapacityIncreaseDecision")
Capacity_Increase_Decision_2 = m.addVars(['Wind', 'Solar', 'Nuclear'],T, vtype=GRB.BINARY, name="CapacityIncreaseDecision2")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in T), index=0, priority=1, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][T.index(t)] * Gen[g, t] for g in G for t in T) +
    sum(Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision[g, t] +Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision_2[g, t] for g in ['Wind', 'Solar', 'Nuclear'] for t in T)  # Fixed opening cost
)
m.setObjectiveN(Total_Cost, index=1, priority=0, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in G) <= Goals[t] + Deviation[t] for t in T), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][T.index(t)] * Gen[g, t] for g in G for t in T), "EmissionCalculation")

# Demand satisfaction
for t in T:
    m.addConstr(sum(Gen[g, t] for g in G) >= D_data[t], "DemandSatisfaction_" + str(t))

#fixed capacity limits for all sources in 2025
for g in G:
    m.addConstr(Gen[g, 2025] <= Fixed_GEN_data[g][T.index(2025)], f"FixedCapacityLimit_{g}_2025")

# Adjusted capacity limits for 2030
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider added capacity in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030], f"AdjustedCapacityLimit_{g}_2030")
    else:
        # For other sources, use fixed capacity limits in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2030")

# Adjusted capacity limits for 2035, considering the capacity from 2030 and additional increase in 2035
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider the total capacity (2030 increase + 2035 increase)
        total_capacity_2035 = Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030] + Capacity_Increase[g, 2035]
        m.addConstr(Gen[g, 2035] <= total_capacity_2035, f"TotalAdjustedCapacityLimit_{g}_2035")
    else:
        # For other sources, use fixed capacity limits in 2035
        m.addConstr(Gen[g, 2035] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2035")


# Capacity increase constraints
for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:
                m.addConstr(Capacity_Increase[g, t] <= Capacity_Increase_Decision[g, t] * Capacity_Added[g][T.index(t)] +
                Capacity_Increase_Decision_2[g, t] * Capacity_Added[g][T.index(t)] , f"CapacityIncreaseLink_{g}_{t}")



#constraint to make sure powerplants are only added if existing capacity is not enough
utilization_threshold = 1  # Define a threshold for capacity utilization
M = 1e15  

for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:  
        existing_capacity_expr = Fixed_GEN_data[g][0] + sum(Capacity_Increase[g, year] for year in T if year < t)

        # Binary variable indicating if capacity increase is needed
        y_gt = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}")
        m.addConstr(Capacity_Increase_Decision[g, t] + M*y_gt >=1)
        m.addConstr(Gen[g, t] - (utilization_threshold * existing_capacity_expr) - M * (1-y_gt) <= 0, name=f"ThresholdCheck_{g}_{t}")
        
        # Binary variable indicating if second capacity increase is needed
        y_gt2 = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}_2")
        m.addConstr(Capacity_Increase_Decision_2[g, t] + M * y_gt2 >= 1)
        m.addConstr(Gen[g, t] - (utilization_threshold * (existing_capacity_expr + Capacity_Added[g][T.index(t)])) - M * (1 - y_gt2) <= 0, name=f"ThresholdCheck_{g}_{t}_2")
#constraint capacity incease 2025 for renewables is 0
m.addConstr(Capacity_Increase['Wind', 2025] == 0)
m.addConstr(Capacity_Increase['Solar', 2025] == 0)
m.addConstr(Capacity_Increase['Nuclear', 2025] == 0)


# Phase out non-renewables
m.addConstr(Gen['Coal', 2030] == 0, "Coal_PhaseOut_2035")
m.addConstr(Gen['Oil', 2035] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen['Coal', 2035] == 0, "Coal_PhaseOut_2035")




    
    
# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")

        # Initialize variables to store total costs and generations
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        total_capacity_increase_cost_year = 0  # Total cost of adding capacity for the year
        total_generation_cost_year = 0  # Total cost of generating electricity for the year
        deviation_year = Deviation[t].x

        # Print generation, emissions, and cost for each energy source
        for g in G:
            generation = Gen[g, t].x
            emissions = Emissions[g, t].x
            cost = generation * Cost_data[g][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_generation_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Generation Cost ({g}): {cost} CAD")

        # Check and print details for each power plant type opened
        for g in ['Wind', 'Solar', 'Nuclear']:
            # First power plant
            power_plant_opened = Capacity_Increase_Decision[g, t].x
            if power_plant_opened > 0.5:  # Power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No {g} Power Plant Opened in {t}")

            # Second power plant
            power_plant_opened_2 = Capacity_Increase_Decision_2[g, t].x
            if power_plant_opened_2 > 0.5:  # Second power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Second Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No Second {g} Power Plant Opened in {t}")

        # Calculate and print total available capacity for Wind, Solar, and Nuclear
        for g in ['Wind', 'Solar', 'Nuclear']:
            total_capacity = Fixed_GEN_data[g][0]
            for year in T:
                if year <= t:
                    added_capacity = Capacity_Increase_Decision[g, year].x * Capacity_Added[g][T.index(year)]
                    added_capacity += Capacity_Increase_Decision_2[g, year].x * Capacity_Added[g][T.index(year)]
                    total_capacity += added_capacity
            print(f"  Total Available Capacity for {g} in {t}: {total_capacity} GWh")

        # Print total costs and generations
        total_cost_year = total_generation_cost_year + total_capacity_increase_cost_year
        print(f"Total Generation Cost for Year {t}: {total_generation_cost_year} CAD")
        print(f"Total Capacity Increase Cost for Year {t}: {total_capacity_increase_cost_year} CAD")
        print(f"Total Cost for Year {t}: {total_cost_year} CAD")
        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")





Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 90 rows, 90 columns and 222 nonzeros
Model fingerprint: 0x79bf2654
Variable types: 60 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+15]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 48 rows and 30 columns
Presolve time: 

Multi-objectives: optimize objective 1 (Minimize_Emissions) ...
---------------------------------------------------------------------------

Found heuristic solution: objective 150.6183069
Presolve removed 40 rows and 53 columns
Presolve time: 0.00s
Presolved: 2 rows, 7 columns, 13 nonzeros
Found heuristic solution: objective 74.2648242
Variable types: 7 continuous, 0 integer (0 binary)

Root relaxation: objective 1.349566e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      13.4956649   13.49566  0.00%     -    0s

Explored 1 nodes (7 simplex iterations) in 0.17 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 13.4957 150.618 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.349566488000e+01, best bound 1.349566488000e+01, gap 0.0000%
---------------

## Printing all the Decision Variables

In [17]:
for v in m.getVars():
    print(v.varName, v.x)

Generation[Wind,2025] 64389.48
Generation[Wind,2030] 151989.48000000004
Generation[Wind,2035] 239589.48000000004
Generation[Solar,2025] 11402.425080834817
Generation[Solar,2030] 0.0
Generation[Solar,2035] 0.0
Generation[Oil,2025] 1379.91
Generation[Oil,2030] 1379.91
Generation[Oil,2035] 0.0
Generation[Nuclear,2025] 78631.37
Generation[Nuclear,2030] 130180.1899999999
Generation[Nuclear,2035] 271351.37
Generation[Hydro,2025] 402575.9
Generation[Hydro,2030] 402575.9
Generation[Hydro,2035] 276900.2599999999
Generation[Natural Gas,2025] 45146.6549191651
Generation[Natural Gas,2030] 0.0
Generation[Natural Gas,2035] 0.0
Generation[Coal,2025] 0.0
Generation[Coal,2030] 0.0
Generation[Coal,2035] 0.0
Generation[Geothermal,2025] 8281.2
Generation[Geothermal,2030] 8281.2
Generation[Geothermal,2035] 0.0
Emissions[Wind,2025] 0.9658422000000001
Emissions[Wind,2030] 2.2798422000000005
Emissions[Wind,2035] 3.5938422000000005
Emissions[Solar,2025] 0.5074079160971493
Emissions[Solar,2030] 0.0
Emissions[So

## Sensitivity Analysis

#### Which constraints are binding?

In [18]:
import pandas as pd

# Prepare lists to hold constraint information
constraint_names = []
constraint_sense = []
constraint_slack = []
constraint_rhs = []

for constr in m.getConstrs():
    constraint_names.append(constr.ConstrName)
    constraint_sense.append(constr.Sense)
    constraint_slack.append(constr.Slack)
    constraint_rhs.append(constr.RHS)

constraints_df = pd.DataFrame({
    'Sense': constraint_sense,
    'Slack': constraint_slack,
    'RHS': constraint_rhs
},index=constraint_names)

# Display the DataFrame showing all the rows
pd.set_option('display.max_rows', None)
constraints_df


,Sense,Slack,RHS
EmissionGoals[2025],<,0.000000e+00,3.519000e+01
EmissionGoals[2030],<,1.439533e+01,2.889000e+01
EmissionGoals[2035],<,-4.796163e-14,0.000000e+00
"EmissionCalculation[Wind,2025]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Wind,2030]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Wind,2035]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2025]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2030]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2035]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Oil,2025]",=,0.000000e+00,0.000000e+00


## Demand Uncertainty

#### How does increase in demand affect the model?


- The model is very sensitive to increase in demand. The emission deviation in 2025 increases with even 1% increase in demand. Interestingly, increase in demand in 2025 reduces the emission deviation probably because the model needs to meet the demand by allocating more resources from renewable sources regardless of the cost



In [19]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal', 'Geothermal']

# Emission factors for each energy source (MTCO2e/GWh) 
E_factor_data = {
    'Wind': [0.000015, 0.000015, 0.000015],
    'Solar': [0.0000445, 0.0000445, 0.0000445],
    'Oil': [0.00049, 0.00049, 0.00049],
    'Nuclear': [0.000012, 0.000012, 0.000012],
    'Hydro': [0.000024, 0.000024, 0.000024],
    'Natural Gas': [0.00049, 0.00049, 0.00049],
    'Coal': [0.00082, 0.00082, 0.00082],
    'Geothermal': [0.000038, 0.000038, 0.000038]
}
# Electricity demand for each time period (GWh) 
D_data = {2025: 611806.94, 2030: 694406.68, 2035: 787841.11} 

# Emission reduction  for each time period (#MTCO2e) 
Goals = {2025: 35.19, 2030: 28.89, 2035: 0}  


# Fixed generation capacity for each time period (GWh) 
Fixed_GEN_data = {
    'Wind': [64389.48],
    'Solar': [12184.92],
    'Oil': [1379.91],
    'Nuclear': [78631.37],
    'Hydro': [402575.9],
    'Natural Gas': [106529.07],
    'Coal': [8184.54],
    'Geothermal': [8281.2]
}


# Cost data for each energy source (CAD per GWh) 
Cost_data = {
    'Wind': [80830, 80830, 80830],
    'Solar': [205500, 205500, 205500],
    'Oil': [61650, 61650, 61650],
    'Nuclear': [89050, 89050, 89050],
    'Hydro': [30140, 30140, 30140],
    'Natural Gas': [97270, 97270, 97270],
    'Coal': [83570, 83570, 83570],
    'Geothermal': [82200, 82200, 82200]
}

# Capacity increase costs for each power plant type (CAD)
Increase_Cost = {
    'Nuclear': [9.59e9, 9.59e9, 9.59e9],
    'Solar': [12.878e9, 12.878e9, 12.878e9],
    'Wind': [4.11e9, 4.11e9, 4.11e9]
}

# Capacity increase amounts for each power plant type (GWh)
Capacity_Added = {
    'Nuclear': [48180, 48180, 48180],
    'Solar': [139809.6, 139809.6, 139809.6],
    'Wind': [43800, 43800, 43800]
}

# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(T, vtype=GRB.CONTINUOUS, name="Deviation")

# Decision variables for capacity increase (only for Wind, Solar, Nuclear)
Capacity_Increase = m.addVars(['Wind', 'Solar', 'Nuclear'], [2030,2035], vtype=GRB.CONTINUOUS, name="Capacity_Increase")

# Binary variables for capacity increase decision
Capacity_Increase_Decision = m.addVars(['Wind', 'Solar', 'Nuclear'],T,vtype=GRB.BINARY, name="CapacityIncreaseDecision")
Capacity_Increase_Decision_2 = m.addVars(['Wind', 'Solar', 'Nuclear'],T, vtype=GRB.BINARY, name="CapacityIncreaseDecision2")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in T), index=0, priority=1, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][T.index(t)] * Gen[g, t] for g in G for t in T) +
    sum(Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision[g, t] +Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision_2[g, t] for g in ['Wind', 'Solar', 'Nuclear'] for t in T)  # Fixed opening cost
)
m.setObjectiveN(Total_Cost, index=1, priority=0, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in G) <= Goals[t] + Deviation[t] for t in T), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][T.index(t)] * Gen[g, t] for g in G for t in T), "EmissionCalculation")

# Demand satisfaction
for t in T:
    m.addConstr(sum(Gen[g, t] for g in G) >= D_data[t]+0.12 * D_data[t], "DemandSatisfaction_" + str(t))

#fixed capacity limits for all sources in 2025
for g in G:
    m.addConstr(Gen[g, 2025] <= Fixed_GEN_data[g][T.index(2025)], f"FixedCapacityLimit_{g}_2025")

# Adjusted capacity limits for 2030
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider added capacity in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030], f"AdjustedCapacityLimit_{g}_2030")
    else:
        # For other sources, use fixed capacity limits in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2030")

# Adjusted capacity limits for 2035, considering the capacity from 2030 and additional increase in 2035
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider the total capacity (2030 increase + 2035 increase)
        total_capacity_2035 = Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030] + Capacity_Increase[g, 2035]
        m.addConstr(Gen[g, 2035] <= total_capacity_2035, f"TotalAdjustedCapacityLimit_{g}_2035")
    else:
        # For other sources, use fixed capacity limits in 2035
        m.addConstr(Gen[g, 2035] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2035")


# Capacity increase constraints
for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:
                m.addConstr(Capacity_Increase[g, t] <= Capacity_Increase_Decision[g, t] * Capacity_Added[g][T.index(t)] +
                Capacity_Increase_Decision_2[g, t] * Capacity_Added[g][T.index(t)] , f"CapacityIncreaseLink_{g}_{t}")



#constraint to make sure powerplant only added if existing capacity is not enough
utilization_threshold = 1  # Define a threshold for capacity utilization
M = 1e15  

for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:  
        existing_capacity_expr = Fixed_GEN_data[g][0] + sum(Capacity_Increase[g, year] for year in [2030,2035] if year < t)

        # Binary variable indicating if capacity increase is needed
        y_gt = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}")
        m.addConstr(Capacity_Increase_Decision[g, t] + M*y_gt >=1)
        m.addConstr(Gen[g, t] - (utilization_threshold * existing_capacity_expr) - M * (1-y_gt) <= 0, name=f"ThresholdCheck_{g}_{t}")
        
        # Binary variable indicating if second capacity increase is needed
        y_gt2 = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}_2")
        m.addConstr(Capacity_Increase_Decision_2[g, t] + M * y_gt2 >= 1)
        m.addConstr(Gen[g, t] - (utilization_threshold * (existing_capacity_expr + Capacity_Added[g][T.index(t)])) - M * (1 - y_gt2) <= 0, name=f"ThresholdCheck_{g}_{t}_2")



# Phase out non-renewables
m.addConstr(Gen['Coal', 2030] == 0, "Coal_PhaseOut_2035")
m.addConstr(Gen['Oil', 2035] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen['Coal', 2035] == 0, "Coal_PhaseOut_2035")




    
    
# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")

        # Initialize variables to store total costs and generations
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        total_capacity_increase_cost_year = 0  # Total cost of adding capacity for the year
        total_generation_cost_year = 0  # Total cost of generating electricity for the year
        deviation_year = Deviation[t].x

        # Print generation, emissions, and cost for each energy source
        for g in G:
            generation = Gen[g, t].x
            emissions = Emissions[g, t].x
            cost = generation * Cost_data[g][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_generation_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Generation Cost ({g}): {cost} CAD")

        # Check and print details for each power plant type opened
        for g in ['Wind', 'Solar', 'Nuclear']:
            # First power plant
            power_plant_opened = Capacity_Increase_Decision[g, t].x
            if power_plant_opened > 0.5:  # Power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No {g} Power Plant Opened in {t}")

            # Second power plant
            power_plant_opened_2 = Capacity_Increase_Decision_2[g, t].x
            if power_plant_opened_2 > 0.5:  # Second power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Second Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No Second {g} Power Plant Opened in {t}")

        # Calculate and print total available capacity for Wind, Solar, and Nuclear
        for g in ['Wind', 'Solar', 'Nuclear']:
            total_capacity = Fixed_GEN_data[g][0]
            for year in T:
                if year <= t:
                    added_capacity = Capacity_Increase_Decision[g, year].x * Capacity_Added[g][T.index(year)]
                    added_capacity += Capacity_Increase_Decision_2[g, year].x * Capacity_Added[g][T.index(year)]
                    total_capacity += added_capacity
            print(f"  Total Available Capacity for {g} in {t}: {total_capacity} GWh")

        # Print total costs and generations
        total_cost_year = total_generation_cost_year + total_capacity_increase_cost_year
        print(f"Total Generation Cost for Year {t}: {total_generation_cost_year} CAD")
        print(f"Total Capacity Increase Cost for Year {t}: {total_capacity_increase_cost_year} CAD")
        print(f"Total Cost for Year {t}: {total_cost_year} CAD")
        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")





Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 87 rows, 87 columns and 207 nonzeros
Model fingerprint: 0x285b1983
Variable types: 57 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+15]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 42 rows and 24 columns
Presolve time: 

## Cost of not Satisfying Electricity Demand in Each Year
- The model chooses to pay the penalty associated with not meeting demand in 2035 regardless of how high the penalty is. This is probably because the first priority is to minimize the deviation from zero emission as much as possible in 2035. This changed when the approach was changed from hierarchical to weighted sum with both objectives having equal weigths. 

In [20]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal', 'Geothermal']

# Emission factors for each energy source (MTCO2e/GWh) 
E_factor_data = {
    'Wind': [0.000015, 0.000015, 0.000015],
    'Solar': [0.0000445, 0.0000445, 0.0000445],
    'Oil': [0.00049, 0.00049, 0.00049],
    'Nuclear': [0.000012, 0.000012, 0.000012],
    'Hydro': [0.000024, 0.000024, 0.000024],
    'Natural Gas': [0.00049, 0.00049, 0.00049],
    'Coal': [0.00082, 0.00082, 0.00082],
    'Geothermal': [0.000038, 0.000038, 0.000038]
}


# Electricity demand for each time period (GWh) 
D_data = {2025: 611806.94, 2030: 694406.68, 2035: 787841.11} 

# Emission reduction  for each time period (#MTCO2e) 
Goals = {2025: 35.19, 2030: 28.89, 2035: 0}  #Mahrukh


# Fixed generation capacity for each time period (GWh)  
Fixed_GEN_data = {
    'Wind': [64389.48],
    'Solar': [12184.92],
    'Oil': [1379.91],
    'Nuclear': [78631.37],
    'Hydro': [402575.9],
    'Natural Gas': [106529.07],
    'Coal': [8184.54],
    'Geothermal': [8281.2]
}


# Cost data for each energy source (CAD per GWh) 
Cost_data = {
    'Wind': [80830, 80830, 80830],
    'Solar': [205500, 205500, 205500],
    'Oil': [61650, 61650, 61650],
    'Nuclear': [89050, 89050, 89050],
    'Hydro': [30140, 30140, 30140],
    'Natural Gas': [97270, 97270, 97270],
    'Coal': [83570, 83570, 83570],
    'Geothermal': [82200, 82200, 82200]
}

# Capacity increase costs for each power plant type (CAD)
Increase_Cost = {
    'Nuclear': [9.59e9, 9.59e9, 9.59e9],
    'Solar': [12.878e9, 12.878e9, 12.878e9],
    'Wind': [4.11e9, 4.11e9, 4.11e9]
}

# Capacity increase amounts for each power plant type (GWh)
Capacity_Added = {
    'Nuclear': [48180, 48180, 48180],
    'Solar': [139809.6, 139809.6, 139809.6],
    'Wind': [43800, 43800, 43800]
}
pen=183000
# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(T, vtype=GRB.CONTINUOUS, name="Deviation")

# Decision variables for capacity increase (only for Wind, Solar, Nuclear)
Capacity_Increase = m.addVars(['Wind', 'Solar', 'Nuclear'], [2030,2035], vtype=GRB.CONTINUOUS, name="Capacity_Increase")

# Binary variables for capacity increase decision
Capacity_Increase_Decision = m.addVars(['Wind', 'Solar', 'Nuclear'],T,vtype=GRB.BINARY, name="CapacityIncreaseDecision")
Capacity_Increase_Decision_2 = m.addVars(['Wind', 'Solar', 'Nuclear'],T, vtype=GRB.BINARY, name="CapacityIncreaseDecision2")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in T), index=0, weight=1, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][T.index(t)] * Gen[g, t] for g in G for t in T) +
    pen*sum((D_data[t] - sum(Gen[g, t] for g in G)) for t in T) +
    sum(Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision[g, t] +Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision_2[g, t] for g in ['Wind', 'Solar', 'Nuclear'] for t in T)  # Fixed opening cost
)
m.setObjectiveN(Total_Cost, index=1, weight=1, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in G) <= Goals[t] + Deviation[t] for t in T), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][T.index(t)] * Gen[g, t] for g in G for t in T), "EmissionCalculation")

# Demand satisfaction
for t in T:
    m.addConstr(sum(Gen[g, t] for g in G) <= D_data[t], "DemandSatisfaction_" + str(t))

#fixed capacity limits for all sources in 2025
for g in G:
    m.addConstr(Gen[g, 2025] <= Fixed_GEN_data[g][T.index(2025)], f"FixedCapacityLimit_{g}_2025")

# Adjusted capacity limits for 2030
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider added capacity in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030], f"AdjustedCapacityLimit_{g}_2030")
    else:
        # For other sources, use fixed capacity limits in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2030")

# Adjusted capacity limits for 2035, considering the capacity from 2030 and additional increase in 2035
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider the total capacity (2030 increase + 2035 increase)
        total_capacity_2035 = Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030] + Capacity_Increase[g, 2035]
        m.addConstr(Gen[g, 2035] <= total_capacity_2035, f"TotalAdjustedCapacityLimit_{g}_2035")
    else:
        # For other sources, use fixed capacity limits in 2035
        m.addConstr(Gen[g, 2035] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2035")


# Capacity increase constraints
for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:
                m.addConstr(Capacity_Increase[g, t] <= Capacity_Increase_Decision[g, t] * Capacity_Added[g][T.index(t)] +
                Capacity_Increase_Decision_2[g, t] * Capacity_Added[g][T.index(t)] , f"CapacityIncreaseLink_{g}_{t}")



#constraint to make sure powerplant only added if existing capacity is not enough
utilization_threshold = 1  # Define a threshold for capacity utilization
M = 1e15  

for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:  
        existing_capacity_expr = Fixed_GEN_data[g][0] + sum(Capacity_Increase[g, year] for year in [2030,2035] if year < t)

        # Binary variable indicating if capacity increase is needed
        y_gt = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}")
        m.addConstr(Capacity_Increase_Decision[g, t] + M*y_gt >=1)
        m.addConstr(Gen[g, t] - (utilization_threshold * existing_capacity_expr) - M * (1-y_gt) <= 0, name=f"ThresholdCheck_{g}_{t}")
        
        # Binary variable indicating if second capacity increase is needed
        y_gt2 = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}_2")
        m.addConstr(Capacity_Increase_Decision_2[g, t] + M * y_gt2 >= 1)
        m.addConstr(Gen[g, t] - (utilization_threshold * (existing_capacity_expr + Capacity_Added[g][T.index(t)])) - M * (1 - y_gt2) <= 0, name=f"ThresholdCheck_{g}_{t}_2")



# Phase out non-renewables
m.addConstr(Gen['Coal', 2030] == 0, "Coal_PhaseOut_2035")
m.addConstr(Gen['Oil', 2035] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen['Coal', 2035] == 0, "Coal_PhaseOut_2035")


    
    
# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")

        # Initialize variables to store total costs and generations
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        total_capacity_increase_cost_year = 0  # Total cost of adding capacity for the year
        total_generation_cost_year = 0  # Total cost of generating electricity for the year
        deviation_year = Deviation[t].x

        # Print generation, emissions, and cost for each energy source
        for g in G:
            generation = Gen[g, t].x
            emissions = Emissions[g, t].x
            cost = generation * Cost_data[g][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_generation_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Generation Cost ({g}): {cost} CAD")

        # Check and print details for each power plant type opened
        for g in ['Wind', 'Solar', 'Nuclear']:
            # First power plant
            power_plant_opened = Capacity_Increase_Decision[g, t].x
            if power_plant_opened > 0.5:  # Power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No {g} Power Plant Opened in {t}")

            # Second power plant
            power_plant_opened_2 = Capacity_Increase_Decision_2[g, t].x
            if power_plant_opened_2 > 0.5:  # Second power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Second Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No Second {g} Power Plant Opened in {t}")

        # Calculate and print total available capacity for Wind, Solar, and Nuclear
        for g in ['Wind', 'Solar', 'Nuclear']:
            total_capacity = Fixed_GEN_data[g][0]
            for year in T:
                if year <= t:
                    added_capacity = Capacity_Increase_Decision[g, year].x * Capacity_Added[g][T.index(year)]
                    added_capacity += Capacity_Increase_Decision_2[g, year].x * Capacity_Added[g][T.index(year)]
                    total_capacity += added_capacity
            print(f"  Total Available Capacity for {g} in {t}: {total_capacity} GWh")

        # Print total costs and generations
        total_cost_year = total_generation_cost_year + total_capacity_increase_cost_year
        print(f"Total Generation Cost for Year {t}: {total_generation_cost_year} CAD")
        print(f"Total Capacity Increase Cost for Year {t}: {total_capacity_increase_cost_year} CAD")
        print(f"Total Cost for Year {t}: {total_cost_year} CAD")
        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")
# print demand not satisfied
for t in T:
    print(f"Year {t}:")
    print(f"  Demand Not Satisfied: {D_data[t] - sum(Gen[g, t].x for g in G)} GWh")



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 87 rows, 87 columns and 207 nonzeros
Model fingerprint: 0x9431d02d
Variable types: 57 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+15]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
--------------------------------------

         to avoid numerical issues.
Found heuristic solution: objective 3.832120e+11
Presolve removed 76 rows and 69 columns
Presolve time: 0.00s
Presolved: 11 rows, 18 columns, 44 nonzeros
Found heuristic solution: objective 2.364736e+11
Variable types: 10 continuous, 8 integer (8 binary)

Root relaxation: objective 1.253371e+11, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2534e+11    0    1 2.3647e+11 1.2534e+11  47.0%     -    0s
H    0     0                    1.256691e+11 1.2534e+11  0.26%     -    0s
H    0     0                    1.256440e+11 1.2534e+11  0.24%     -    0s
     0     0     cutoff    0      1.2564e+11 1.2564e+11  0.00%     -    0s

Cutting planes:
  Relax-and-lift: 1

Explored 1 nodes (11 simplex iterations) in 0.29 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution 

## Weighted Sum Approach


In [21]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal', 'Geothermal']

# Emission factors for each energy source (MTCO2e/GWh) 
E_factor_data = {
    'Wind': [0.000015, 0.000015, 0.000015],
    'Solar': [0.0000445, 0.0000445, 0.0000445],
    'Oil': [0.00049, 0.00049, 0.00049],
    'Nuclear': [0.000012, 0.000012, 0.000012],
    'Hydro': [0.000024, 0.000024, 0.000024],
    'Natural Gas': [0.00049, 0.00049, 0.00049],
    'Coal': [0.00082, 0.00082, 0.00082],
    'Geothermal': [0.000038, 0.000038, 0.000038]
}

# Electricity demand for each time period (GWh) 
D_data = {2025: 611806.94, 2030: 694406.68, 2035: 787841.11} 

# Emission reduction  for each time period (#MTCO2e) 
Goals = {2025: 35.19, 2030: 28.89, 2035: 0}  


# Fixed generation capacity for each time period (GWh)  
Fixed_GEN_data = {
    'Wind': [64389.48],
    'Solar': [12184.92],
    'Oil': [1379.91],
    'Nuclear': [78631.37],
    'Hydro': [402575.9],
    'Natural Gas': [106529.07],
    'Coal': [8184.54],
    'Geothermal': [8281.2]
}


# Cost data for each energy source (CAD per GWh) 
Cost_data = {
    'Wind': [80830, 80830, 80830],
    'Solar': [205500, 205500, 205500],
    'Oil': [61650, 61650, 61650],
    'Nuclear': [89050, 89050, 89050],
    'Hydro': [30140, 30140, 30140],
    'Natural Gas': [97270, 97270, 97270],
    'Coal': [83570, 83570, 83570],
    'Geothermal': [82200, 82200, 82200]
}

# Capacity increase costs for each power plant type (CAD)
Increase_Cost = {
    'Nuclear': [9.59e9, 9.59e9, 9.59e9],
    'Solar': [12.878e9, 12.878e9, 12.878e9],
    'Wind': [4.11e9, 4.11e9, 4.11e9]
}

# Capacity increase amounts for each power plant type (GWh)
Capacity_Added = {
    'Nuclear': [48180, 48180, 48180],
    'Solar': [139809.6, 139809.6, 139809.6],
    'Wind': [43800, 43800, 43800]
}

# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(T, vtype=GRB.CONTINUOUS, name="Deviation")

# Decision variables for capacity increase (only for Wind, Solar, Nuclear)
Capacity_Increase = m.addVars(['Wind', 'Solar', 'Nuclear'], T, vtype=GRB.CONTINUOUS, name="Capacity_Increase")

# Binary variables for capacity increase decision
Capacity_Increase_Decision = m.addVars(['Wind', 'Solar', 'Nuclear'],T,vtype=GRB.BINARY, name="CapacityIncreaseDecision")
Capacity_Increase_Decision_2 = m.addVars(['Wind', 'Solar', 'Nuclear'],T, vtype=GRB.BINARY, name="CapacityIncreaseDecision2")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in T), index=0, weight=0.5, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][T.index(t)] * Gen[g, t] for g in G for t in T) +
    sum(Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision[g, t] +Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision_2[g, t] for g in ['Wind', 'Solar', 'Nuclear'] for t in T)  # Fixed opening cost
)
m.setObjectiveN(Total_Cost, index=1, weight=0.5, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in G) <= Goals[t] + Deviation[t] for t in T), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][T.index(t)] * Gen[g, t] for g in G for t in T), "EmissionCalculation")

# Demand satisfaction
for t in T:
    m.addConstr(sum(Gen[g, t] for g in G) >= D_data[t], "DemandSatisfaction_" + str(t))

#fixed capacity limits for all sources in 2025
for g in G:
    m.addConstr(Gen[g, 2025] <= Fixed_GEN_data[g][T.index(2025)], f"FixedCapacityLimit_{g}_2025")

# Adjusted capacity limits for 2030
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider added capacity in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030], f"AdjustedCapacityLimit_{g}_2030")
    else:
        # For other sources, use fixed capacity limits in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2030")

# Adjusted capacity limits for 2035, considering the capacity from 2030 and additional increase in 2035
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider the total capacity (2030 increase + 2035 increase)
        total_capacity_2035 = Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030] + Capacity_Increase[g, 2035]
        m.addConstr(Gen[g, 2035] <= total_capacity_2035, f"TotalAdjustedCapacityLimit_{g}_2035")
    else:
        # For other sources, use fixed capacity limits in 2035
        m.addConstr(Gen[g, 2035] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2035")


# Capacity increase constraints
for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:
                m.addConstr(Capacity_Increase[g, t] <= Capacity_Increase_Decision[g, t] * Capacity_Added[g][T.index(t)] +
                Capacity_Increase_Decision_2[g, t] * Capacity_Added[g][T.index(t)] , f"CapacityIncreaseLink_{g}_{t}")



#constraint to make sure powerplant only added if existing capacity is not enough
utilization_threshold = 1  # Define a threshold for capacity utilization
M = 1e15  

for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:  
        existing_capacity_expr = Fixed_GEN_data[g][0] + sum(Capacity_Increase[g, year] for year in T if year < t)

        # Binary variable indicating if capacity increase is needed
        y_gt = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}")
        m.addConstr(Capacity_Increase_Decision[g, t] + M*y_gt >=1)
        m.addConstr(Gen[g, t] - (utilization_threshold * existing_capacity_expr) - M * (1-y_gt) <= 0, name=f"ThresholdCheck_{g}_{t}")
        
        # Binary variable indicating if second capacity increase is needed
        y_gt2 = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}_2")
        m.addConstr(Capacity_Increase_Decision_2[g, t] + M * y_gt2 >= 1)
        m.addConstr(Gen[g, t] - (utilization_threshold * (existing_capacity_expr + Capacity_Added[g][T.index(t)])) - M * (1 - y_gt2) <= 0, name=f"ThresholdCheck_{g}_{t}_2")
#constraint capacity incease 2025 for renewables is 0
m.addConstr(Capacity_Increase['Wind', 2025] == 0)
m.addConstr(Capacity_Increase['Solar', 2025] == 0)
m.addConstr(Capacity_Increase['Nuclear', 2025] == 0)


# Phase out non-renewables
m.addConstr(Gen['Coal', 2030] == 0, "Coal_PhaseOut_2035")
m.addConstr(Gen['Oil', 2035] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen['Coal', 2035] == 0, "Coal_PhaseOut_2035")



    
    
# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")

        # Initialize variables to store total costs and generations
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        total_capacity_increase_cost_year = 0  # Total cost of adding capacity for the year
        total_generation_cost_year = 0  # Total cost of generating electricity for the year
        deviation_year = Deviation[t].x

        # Print generation, emissions, and cost for each energy source
        for g in G:
            generation = Gen[g, t].x
            emissions = Emissions[g, t].x
            cost = generation * Cost_data[g][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_generation_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Generation Cost ({g}): {cost} CAD")

        # Check and print details for each power plant type opened
        for g in ['Wind', 'Solar', 'Nuclear']:
            # First power plant
            power_plant_opened = Capacity_Increase_Decision[g, t].x
            if power_plant_opened > 0.5:  # Power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No {g} Power Plant Opened in {t}")

            # Second power plant
            power_plant_opened_2 = Capacity_Increase_Decision_2[g, t].x
            if power_plant_opened_2 > 0.5:  # Second power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Second Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No Second {g} Power Plant Opened in {t}")

        # Calculate and print total available capacity for Wind, Solar, and Nuclear
        for g in ['Wind', 'Solar', 'Nuclear']:
            total_capacity = Fixed_GEN_data[g][0]
            for year in T:
                if year <= t:
                    added_capacity = Capacity_Increase_Decision[g, year].x * Capacity_Added[g][T.index(year)]
                    added_capacity += Capacity_Increase_Decision_2[g, year].x * Capacity_Added[g][T.index(year)]
                    total_capacity += added_capacity
            print(f"  Total Available Capacity for {g} in {t}: {total_capacity} GWh")

        # Print total costs and generations
        total_cost_year = total_generation_cost_year + total_capacity_increase_cost_year
        print(f"Total Generation Cost for Year {t}: {total_generation_cost_year} CAD")
        print(f"Total Capacity Increase Cost for Year {t}: {total_capacity_increase_cost_year} CAD")
        print(f"Total Cost for Year {t}: {total_cost_year} CAD")
        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")





Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 90 rows, 90 columns and 222 nonzeros
Model fingerprint: 0x3a81ce4c
Variable types: 60 continuous, 30 integer (30 binary)
Coefficient statistics:


  Matrix range     [1e-05, 1e+15]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 90 rows, 90 columns and 222 nonzeros
Model fingerprint: 0x3b9db92b
Variable types: 60 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+15]
  Objective range  [5e-01, 6e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider

## Goal Programming Model

In [22]:
import gurobipy as gb
from gurobipy import *

# Time periods and energy sources
Time = [2025, 2030, 2035]
Gen_tech= ["wind", "solar", "hydro", "nuclear", "natural_gas", "geothermal", "oil", "coal"]

# Generation goals for each source in each time period
Lgt= [
    [60429.05, 92547.75, 184305.21],  # Wind
    [11707.13, 15214.42, 42161.81],   # Solar
    [401733.6, 443103.1, 465887.5],   # Hydro
    [78278.04, 87724.81, 131692.29],  # Nuclear
    [87486.85, 77098.86, 41866.65],   # Natural Gas
    [8281.2, 7526.04, 19716.07],      # Geothermal
    [1355.5, 969.57, 726.09],         # Oil
    [3044.54, 0, 0]                   # Coal
]

# Demand, emission,cost goals, and emission factors
Dt= [668045.91, 770028.33, 886355.62]
It = [30568530907.76, 64177279305.40, 79264644979.28]
Mt= [35.19, 24.89, 0]

Egt = [
    [0.000015, 0.000015, 0.000015],  # Wind
    [0.0000445, 0.0000445, 0.0000445],   # Solar
    [0.000024, 0.000024, 0.000024],   # Hydro
    [0.000012, 0.000012, 0.000012],  # Nuclear
    [0.00049, 0.00049, 0.00049],   # Natural Gas
    [0.000038, 0.000038, 0.000038],      # Geothermal
    [0.00049, 0.00049, 0.00049],         # Oil
    [0.00082, 0.00082, 0.00082]                   # Coal
]

# Emission data for each source in each time period
Bgt = [
    [0.91, 1.39, 2.76],  # Wind
    [0.52, 0.68, 1.88],   # Solar
    [9.64, 10.63, 11.18],   # Hydro
    [0.94, 1.05, 1.58],  # Nuclear
    [42.67, 37.78, 20.51],   # Natural Gas
    [0.31, 0.29, 0.75],      # Geothermal
    [0.66, 0.48, 0.36],         # Oil
    [2.50, 0, 0]                   # Coal
]

# Cost data for each source in each time period
Kgt = [
    [4884480111.50, 7480634632.50, 14897390124.30],  # Wind
    [2405815215.00, 3126563310.00, 8664251955.00],   # Solar
    [24766876440.00, 27317306115.00, 28721964375.00],   # Hydro
    [6970659462.00, 7811894330.50, 11727198424.50],  # Nuclear
    [2636853659.00, 2323759640.40, 1261860831.00],   # Natural Gas
    [805512324.00, 732057910.80, 1917782128.90],      # Geothermal
    [113279135.00, 81026964.9, 60679341.3],         # Oil
    [250261188, 0, 0]                   # Coal
]

# Cost UB for each source in each time period
Kgt_UB = [
    [4884480111.50 * 1.05, 7480634632.50 * 1.05, 14897390124.30 * 1.05],  # Wind
    [2405815215.00 * 1.05, 3126563310.00 * 1.05, 8664251955.00 * 1.05],   # Solar
    [24766876440.00 * 1.05, 27317306115.00 * 1.05, 28721964375.00 * 1.05],   # Hydro
    [6970659462.00 * 1.05, 7811894330.50 * 1.05, 11727198424.50 * 1.05],  # Nuclear
    [2636853659.00 * 1.05, 2323759640.40 * 1.05, 1261860831.00 * 1.05],   # Natural Gas
    [805512324.00 * 1.05, 732057910.80 * 1.05, 1917782128.90 * 1.05],      # Geothermal
    [113279135.00 * 1.05, 81026964.9 * 1.05, 60679341.3 * 1.05],         # Oil
    [250261188 * 1.05, 0, 0]  # Coal
]

# Emission UB for each source in each time period
Bgt_UB= [
    [60429.05*0.00015*1.05, 92547.75*0.00015*1.05, 184305.21*0.00015*1.05],  # Wind
    [11707.13*0.0000445*1.05, 15214.42*0.0000445*1.05, 42161.81*0.0000445*1.05],   # Solar
    [401733.6*0.000024*1.05, 443103.1* 0.000024*1.05, 465887.5* 0.000024*1.05],   # Hydro
    [78278.04*0.000012*1.05, 87724.81*0.000012*1.05, 131692.29*0.000012*1.05],  # Nuclear
    [87486.85*0.00049*1.05, 77098.86*0.00049*1.05, 41866.65*0.00049*1.05],   # Natural Gas
    [8281.2*0.000038*1.05, 7526.04*0.000038*1.05, 19716.07*0.000038*1.05],      # Geothermal
    [1355.5*0.00049*1.05, 969.57*0.00049*1.05, 726.09*0.00049*1.05],         # Oil
    [3044.54*0.00082*1.05, 0*0.00082*1.05, 0*0.00082*1.05]                   # Coal
]

# Generation UB for each source in each time period
Lgt_UB= [
    [60429.05*1.05, 92547.75*1.05,184305.21*1.05],  # Wind
    [11707.13*1.05, 15214.42*1.05, 42161.81*1.05],   # Solar
    [401733.6*1.05, 443103.1*1.05, 465887.5*1.05],   # Hydro
    [78278.04*1.05, 87724.81*1.05, 131692.29*1.05],  # Nuclear
    [87486.85*1.05, 77098.86*1.05, 41866.65*1.05],   # Natural Gas
    [8281.2*1.05, 7526.04*1.05, 19716.07*1.05],      # Geothermal
    [1355.5*1.05, 969.57*1.05, 726.09*1.05],         # Oil
    [3044.54*1.05, 0*1.05, 0*1.05]                   # Coal
]

# Capacity UB for each source in each time period
Ugt_UB = [
    [64389.48*1.05, (64389.48 + 2 * 64389.48)*1.05, (64389.48 + 4 * 64389.48)*1.05],  # Wind
    [12184.92*1.05, (12184.92 + 2 * 12184.92)*1.05, (12184.92 + 4 * 12184.92)*1.05],  # Solar
    [401733.6*1.05, 401733.6*1.05, 401733.6*1.05],                                # Hydro
    [78631.37*1.05, 78631.37*1.05, 78631.37*1.05],                                # Nuclear
    [106529.07*1.05, 106529.07*1.05, 106529.07*1.05],                             # Natural Gas
    [8281.2*1.05, 8281.2*1.05, 8281.2*1.05],                                      # Geothermal
    [1379.91*1.05, 1379.91*1.05, 1379.91*1.05],                                   # Oil
    [8184.54*1.05, 0*1.05, 0*1.05]                                                # Coal & Coke
]

# Capacity goals 
Ugt = [
    [64389.48, 64389.48 + 2 * 64389.48, 64389.48 + 4 * 64389.48],  # Wind
    [12184.92, 12184.92 + 2 * 12184.92, 12184.92 + 4 * 12184.92],  # Solar
    [401733.6, 401733.6, 401733.6],                                # Hydro
    [78631.37, 78631.37, 78631.37],                                # Nuclear
    [106529.07, 106529.07, 106529.07],                             # Natural Gas
    [8281.2, 8281.2, 8281.2],                                      # Geothermal
    [1379.91, 1379.91, 1379.91],                                   # Oil
    [8184.54, 0, 0]                                                # Coal & Coke
]

# Singular Cost data for each source in each time period
Cgt = [
    [80830, 80830, 80830],    # Wind
    [205500, 205500, 205500], # Solar
    [61650, 61650, 61650],    # Hydro
    [89050, 89050, 89050],    # Nuclear
    [30140, 30140, 30140],    # Natural Gas
    [97270, 97270, 97270],    # Geothermal
    [83570, 83570, 83570],    # Oil
    [82200, 82200, 82200]     # Coal
]

G=range(len(Gen_tech))
T=range(len(Time))

# Create model
prob = gb.Model("CanadaEnergyModel")

# Deviation variables
ct_plus  = prob.addVars(G,T, lb=0, name='Positive Cost Deviation')
ct_minus = prob.addVars(G,T, lb=0, name='Negative Cost Deviation')
et_plus  = prob.addVars(G,T, lb=0, name='Positive Emission Deviation')
et_minus = prob.addVars(G,T, lb=0, name='Negative Emission Deviation')
gt_plus  = prob.addVars(G,T, lb=0, name='Positive Generation Deviation')
gt_minus = prob.addVars(G,T, lb=0, name='Negative Generation Deviation')
qt_plus  = prob.addVars(G,T, lb=0, name='Positive Capacity Deviation')
qt_minus = prob.addVars(G,T, lb=0, name='Negative Capacity Deviation')

for g in G:
    for t in T:
        ct_plus[g, t].setAttr('UB', Kgt_UB[g][t])
        ct_minus[g, t].setAttr('UB', Kgt_UB[g][t])
        et_plus[g, t].setAttr('UB', Bgt_UB[g][t])
        et_minus[g, t].setAttr('UB', Bgt_UB[g][t])
        gt_plus[g, t].setAttr('UB', Lgt_UB[g][t])
        gt_minus[g, t].setAttr('UB', Lgt_UB[g][t])
        qt_plus[g, t].setAttr('UB', Ugt_UB[g][t])
        qt_minus[g, t].setAttr('UB', Ugt_UB[g][t])


# Decision variables
Y = prob.addVars(G, T, lb=0, name='Generation')
X = prob.addVars(G, T, lb=0, name='Capacity')
Z = prob.addVars(G,T, lb=0, name='Emission')
C = prob.addVars(G,T, lb=0, name='Cost')


# Constraints
prob.addConstrs(Lgt[g][t] == Y[g, t] + gt_plus[g, t] - gt_minus[g, t] for g in G for t in T)
prob.addConstrs(Ugt[g][t] == X[g, t] + qt_plus[g, t] - qt_minus[g, t] for g in G for t in T)
prob.addConstrs(Bgt[g][t] == Z[g, t] + et_plus[g, t] - et_minus[g, t] for g in G for t in T)
prob.addConstrs(Kgt[g][t] == C[g, t] + ct_plus[g, t] - ct_minus[g, t] for g in G for t in T)

prob.addConstrs(Y[g, t] <= X[g, t] for g in G for t in T)
prob.addConstrs(Z[g, t] == Egt[g][t] * Y[g, t] for g in G for t in T)
prob.addConstrs(C[g, t] == Cgt[g][t] * X[g, t] for g in G for t in T)

prob.addConstrs(sum(Y[g, t] for g in G) >= Dt[t] for t in T)
prob.addConstrs(sum(C[g, t] for g in G) >= It[t] for t in T)
prob.addConstrs(sum(Z[g, t] for g in G) >= Mt[t] for t in T)

# Objectives
Emission_Deviation = sum((et_plus[g,t] + et_minus[g,t] for g in G for t in T))
Cost_Deviation = sum(ct_plus[g,t] + ct_minus[g,t] for g in G for t in T)
Generation_Deviation = sum(gt_plus[g, t] + gt_minus[g, t] for g in G for t in T)
Capacity_Deviation = sum(qt_plus[g, t] + qt_minus[g, t] for g in G for t in T)

prob.setObjectiveN(Emission_Deviation, index=0, priority=3)
prob.setObjectiveN(Generation_Deviation, index=1, priority=2)
prob.setObjectiveN(Capacity_Deviation, index=2, priority=1)
prob.setObjectiveN(Cost_Deviation, index=3, priority=0)

# Optimize
prob.optimize()

# Print solution
for v in prob.getVars():
    print(f"{v.VarName}: {v.x}")

# Print detailed results
print("\nDetailed Results:\n")
for t_idx, t in enumerate(Time):
    print(f"Time Period: {t}")
    
    for g_idx, g in enumerate(Gen_tech):
        gen = Y[g_idx, t_idx].x
        cap = X[g_idx, t_idx].x
        emission = Z[g_idx, t_idx].x
        cost = C[g_idx, t_idx].x
        gen_target = Lgt[g_idx][t_idx]
        cap_target = Ugt[g_idx][t_idx]
        emission_target = Bgt[g_idx][t_idx]
        cost_target = Kgt[g_idx][t_idx]

        # Calculate deviations
        gen_plus = max(0, gen - gen_target)
        gen_minus = max(0, gen_target - gen)
        cap_plus = max(0, cap - cap_target)
        cap_minus = max(0, cap_target - cap)
        emission_plus = max(0, emission - emission_target)  
        emission_minus = max(0, emission_target - emission)  
        cost_plus = max(0, cost - cost_target)
        cost_minus = max(0, cost_target - cost)

        print(f"  Technology: {g}")
        print(f"    Generation (GWh): {gen}")
        print(f"    Capacity (GWh): {cap}")
        print(f"    Emissions (MtCO2e): {emission}")
        print(f"    Cost (CAD): {cost}")
        print(f"    Positive Generation Deviation: {gen_plus}")
        print(f"    Negative Generation Deviation: {gen_minus}")
        print(f"    Positive Capacity Deviation: {cap_plus}")
        print(f"    Negative Capacity Deviation: {cap_minus}")
        print(f"    Positive Emissions Deviation: {emission_plus}")  
        print(f"    Negative Emissions Deviation: {emission_minus}")  
        print(f"    Positive Cost Deviation: {cost_plus}")  
        print(f"    Negative Cost Deviation: {cost_minus}")

    # Print totals for each decision variable in the current time period
    total_gen_t = sum(Y[g_idx, t_idx].x for g_idx in G)
    total_cap_t = sum(X[g_idx, t_idx].x for g_idx in G)
    total_emission_t = sum(Z[g_idx, t_idx].x for g_idx in G)
    total_cost_t = sum(C[g_idx, t_idx].x for g_idx in G)

    print("  --------------------------------------------------\n")
    print(f"  Total Generation (GWh): {total_gen_t}")
    print(f"  Total Capacity (GWh): {total_cap_t}")
    print(f"  Total Emissions (MtCO2e): {total_emission_t}")
    print(f"  Total Cost (CAD): {total_cost_t}")
    print("--------------------------------------------------\n")
    
# After optimization

# Sum decision variables over G and T dimensions 
total_gen = sum(Y[g, t].x for g in G for t in T)
total_cap = sum(X[g, t].x for g in G for t in T)  
total_emission = sum(Z[g, t].x for g in G for t in T)
total_cost = sum(C[g, t].x for g in G for t in T)

print("\nTotal across all time periods:") 
print(f"Total Generation (GWh): {total_gen}")
print(f"Total Capacity (GWh): {total_cap}")
print(f"Total Emissions (MtCO2e): {total_emission}")  
print(f"Total Cost (CAD): {total_cost}")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 177 rows, 288 columns and 504 nonzeros
Model fingerprint: 0x49eca037
Variable types: 288 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-05, 2e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-01, 3e+10]
  RHS range        [3e-01, 8e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 59 rows and 68 columns
Presolved: 118